BESP

23  04 11

Bimodal cell cycle and size related genes

Alex Casar / David Goode

****

**Cleaning and reducing size of seurat objects**

We will save only 3000 variable features and at most 8000 cells with identified barcode.

## Prepare

We allways need this chunck to use R in colab and mount google drive into the sesion

To use R magic in python colab. In 2023 the new rpy2 package was not able to run magic R, using a lower version soved the problem

In [1]:
!pip install rpy2==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# activate R magic
%load_ext rpy2.ipython
# Load drive to use our files
from google.colab import drive
drive.mount('/content/drive')   # Change path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Reload packages and call libraries

We will always use this chunk to call our libraries

In [3]:
# Copy packages file to use it
%cp /content/drive/MyDrive/MELBOURNEbimodal/LIBRARIES/library13.tar.gz .
!tar xf /content/library13.tar.gz 
!rm /content/library13.tar.gz
%R .libPaths('/content/usr/local/lib/R/site-library')   # Change the library used by R binaries


'/content/...,'/usr/loca...,'/usr/lib/...,'/usr/lib/...


In [4]:
%%R
# Libraries
library(dplyr)  # Object management
library(Seurat)   # Single cell analysis
library(patchwork)  # Plots suplements
library(Matrix)   # To work/preview sparse matrix; Seurat is capable to.
library(vioplot) # Violin pots
library(ggplot2)  # Plots
#library(tidyverse)  # Data mining


library(Rcpp)   # It makes very simple to connect C++ to R
library(magrittr)   # Decrease development time and improve readability and maintainability of code
#library(clustree)   # Visualise Clusterings at Different Resolutions.

library(BiocParallel)   # Parallel evaluation
library(BiocParallel)   # Parallel evaluation
library(scShapes)   # A Statistical Framework for Modeling and Identifying Differential Distributions in Single-cell RNA-sequencing Data


library(BimodalIndex)
#library(SeuratData)   # mechanism for distributing datasets in the form of Seurat objects using R's internal package and data management 
                        #systems. It represents an easy way for users to get access to datasets that are used in the Seurat vignettes.


install.packages("DescTools")
library(DescTools)  

library(pheatmap)   # heatmap + clustering plot

library(viridis)  # Kiut ggplot colors
library(reshape)  # To convert matrix to long format required for ggplot

library(iZID)   # New KS-Test

Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union





    consider that it could be called from a Python process. This
    results in a quasi-obligatory segfault when rpy2 is evaluating
    R code using it. On the hand, rpy2 is accounting for the
    fact that it might already be running embedded in a Python
    process. This is why:
    - Python -> rpy2 -> R -> reticulate: crashes
    - R -> reticulate -> Python -> rpy2: works

    The issue with reticulate is tracked here:
    https://github.com/rstudio/reticulate/issues/208
    





Attaching package: ‘zoo’



    as.Date, as.Date.numeric


(as ‘lib’ is unspecified)







	‘/tmp/Rtmp9sUQ4j/downloaded_packages’


Attaching package: ‘reshape’



    expand



    rename




## Old new

Here we define what type of data we were going to use

At the end we were more interested in the new data since it had different stages of diasease

In [5]:
%%R
OLDnew = "OLD"

In [10]:
%%R
OLDnew = "NEW"

In [6]:
%%R
NORMALIZE <- TRUE

## Definitions

In [7]:
%%R
# Parameters
MAX0 <- 0.10  # Keep genes with a certain number (perc.zero percent) of nonzero entries

# Directories
DATAdir <- "/content/drive/MyDrive/MELBOURNEbimodal/DATAinput/"   # Input files
TEMPdir <- "/content/drive/MyDrive/MELBOURNEbimodal/TEMPORAL/"  # Output temporal files
FINALdir <- "/content/drive/MyDrive/MELBOURNEbimodal/FINALdata/"  # Output of relevant files
IMAGEdir <- "/content/drive/MyDrive/MELBOURNEbimodal/IMAGESout" # Image files





# Statements
set.seed(0xBEEF)
setwd(DATAdir)


# Useful function
`%ni%` <- Negate(`%in%`)

In [8]:
%%R
# Samples metadata: We have two kinds of data: Old data and New data
SAMPLESclass <- list()
SAMPLESclass[["NEW"]] <- data.frame(row.names = c("T0", "M15", "M05", "M02", "M10", "M11", "M07", "M12", "M09"), 
                                    TYPE = c("Baseline", "Vehicle", "Vehicle", "Mid Disease", "Mid Disease", "End Disease", "End Disease", "Post Disease", "Post Disease"),
                                    DAY = c(0, NA, NA, 2,2,5,5,7,7))

SAMPLESclass[["OLD"]] <- data.frame(row.names = c("T0", "M8", "M10"), TYPE = c("Vitro", "Vivo", "Vivo"))



## Cleaning / Splitting / Reducing data and saving it

### Old data

  * 3 samples

  * 2 conditions: *in vitro* (T0); *in vivo* (M8, M10)

  * Both *in vivo* samples were extracted in the same time point from different mices


### New data
  * 9 samples
  * 5 conditions
  * 9 mices

In [11]:
%%R
# Reduce amount of cells and split in vivo samples
TYPEdata <- "CleanRed3000"
if (OLDnew == "OLD"){
    INPUTfiles <- paste(DATAdir, c("KRAS_T0_02_POST-DGE-analysis.rds", 
                "KRAS_M8_cc_regressed_normalised_singlets.rds", 
                "KRAS_M10_cc_regressed_normalised_singlets.rds"), sep = "")
    gc()  # To clean our environment

    # Cutoff of RNA features per cell for each sample: This was defined when we explored the violin plots.
    LOW <- c(2000, 300, 900)
    HIGH <- c(6500, 4500, 6500)


    # Update seurat; RNA default; Remove cells/zeros with a lot of 0's; Change to matrix
    SEUR <- list()  # Final compendium
    i <- 1
    for (FILE in INPUTfiles[c(1,2,3)]){
        print(FILE)
        SAMPLE <- strsplit(FILE, "_")[[1]][[2]]
        DATA <- readRDS(FILE); print(DATA)

        DATA <- UpdateSeuratObject(object = DATA)
        DefaultAssay(DATA) <- "RNA"   # The raw counts
        DATA = subset(DATA, subset = (barcode %ni% c("not.detected")))  # Remove the cells ????????
        DATA <- subset(DATA, subset = nFeature_RNA > LOW[i] & nFeature_RNA < HIGH[i])
        DATA <-  FindVariableFeatures(DATA, selection.method = "vst", nfeatures = 3000)   # We are interested only in variable features
        if (NORMALIZE == TRUE){
            DATA <- NormalizeData(DATA)
            TYPEdata <- "CleanNorm3000"
        }
        DATA <- CreateSeuratObject(GetAssayData(DATA)[VariableFeatures(DATA),], project = "BESPreducedVarFea", meta.data = DATA@meta.data)  # We remove non variable features since it is cheaper in RAM
        # The default is to separate the cell by cluster identity, we put the same identity to plot all cells
        Idents(DATA) <- rep("Cell", ncol(DATA))
        DATA <- FindVariableFeatures(DATA, selection.method = "vst", nfeatures = 3000)
        print(DATA)

        SEUR[[SAMPLE]] <- DATA  # Add to compendium
        rm(DATA)  # Clean the environment
        i <- i+1
        gc()
    }

    # Select a smaller dataset for the in vitro sample since it is huge
    SEUR[[1]] <- SEUR[[1]][, sample(colnames(SEUR[[1]]), size = 8000, replace=F)]

    print(SEUR)
    #saveRDS(SEUR, paste(DATAdir, "BESP-Reduced3000VarFeat-", OLDnew, ".Rds"))

} else if (OLDnew == "NEW"){
    ######## In vitro data ##########
    N <- 6 # We are going to subset N*1000 cells for in vitro sample
    SEUR <- list()  # Where we are going to compend all samples of seurat

    # Read the in vitro data
    DATA <- readRDS(paste(DATAdir, "KRAS_T0_merged_cc_regressed_normalised.rds", sep = "")); print(DATA); gc()
    DefaultAssay(DATA) <- "RNA"

    # Reduce the amount of cells for the in vitro sample because it is huge
    # Since it is huge, we only can subset around 6000 cells: 1000 at once
    RANsel <- sample(colnames(DATA), size = N*1000, replace=F)  # We select the cells to use 
    RANsel <- split(RANsel, 1:N)  # We split it into 1000
    
    print("Reducing amount of cells of in vitro sample")
    SEURl <- list()  # Where we are going to compend all subsets of 1000
    VARfea <- list()
    for (i in 1:N){
        SEURl[[i]] <- DATA[, RANsel[[i]]]
        SEURl[[i]] <- subset(SEURl[[i]], subset = (barcode %ni% c("not.detected"))); gc()
        gc()
    }
    rm(DATA); rm(RANsel); gc()
    DATA <- merge(x = SEURl[[1]], y = SEURl[-1]); gc()  # We merge the seurat objects of 1000
    rm(SEURl)
    DATA <- FindVariableFeatures(DATA, selection.method = "vst", nfeatures = 3000)
    if (NORMALIZE == T){
       DATA <- NormalizeData(DATA)
       TYPEdata <- "CleanNorm3000"
    }
    DATA <- CreateSeuratObject(GetAssayData(DATA)[VariableFeatures(DATA),], project = "BESPreducedVarFea", meta.data = DATA@meta.data)
    print("New")
    print(DATA)
    DATA <- FindVariableFeatures(DATA, selection.method = "vst", nfeatures = 3000)
    print("Var")
    Idents(DATA) <- DATA@meta.data$orig.ident; gc()   # The default is to separate the cell by cluster identity, we change it to plot all cells
    print("Idents")
    SEUR[["T0"]] <- DATA

    rm(DATA); gc()

 
    ###### In vivo data ########
    DATA <- readRDS(paste(DATAdir, "KRAS.disease.rds", sep = "")); ; print(DATA); gc()   # Read
    DefaultAssay(DATA) <- "RNA"   # Default dataset RNA
    DATA = subset(DATA, subset = (barcode %ni% c("not.detected")))  # Remove uninteresting cells
    print(DATA)
    SAMPLESvivo <- row.names(SAMPLESclass[[OLDnew]])
    for (SAMPLE in SAMPLESvivo[-1]){
        print(SAMPLE)
        SEUR[[SAMPLE]] <- subset(DATA, cells = row.names(DATA@meta.data)[grep(SAMPLE, DATA@meta.data$HTO_classification)]); print(DATA)
        SEUR[[SAMPLE]] <- FindVariableFeatures(SEUR[[SAMPLE]], selection.method = "vst", nfeatures = 3000)
        if (NORMALIZE == T){
            SEUR[[SAMPLE]] <- NormalizeData(SEUR[[SAMPLE]])
            TYPEdata <- "CleanNorm3000"
        }
        SEUR[[SAMPLE]] <- CreateSeuratObject(GetAssayData(SEUR[[SAMPLE]])[VariableFeatures(SEUR[[SAMPLE]]),], project = "BESPreducedVarFea", meta.data = SEUR[[SAMPLE]]@meta.data)
        SEUR[[SAMPLE]] <- FindVariableFeatures(SEUR[[SAMPLE]], selection.method = "vst", nfeatures = 3000)
        Idents(SEUR[[SAMPLE]]) <- SEUR[[SAMPLE]]@meta.data$orig.ident; gc()   # The default is to separate the cell by cluster identity, we change it to plot all cells

    }
    print(SEUR)
}
saveRDS(SEUR, paste(DATAdir, OLDnew, "-BESP-", TYPEdata, "-AllSamSeu.Rds", sep = ""))

An object of class Seurat 
33971 features across 25557 samples within 2 assays 
Active assay: SCT (16443 features, 3000 variable features)
 1 other assay present: RNA
 2 dimensional reductions calculated: pca, umap50
[1] "Reducing amount of cells of in vitro sample"


[1] "New"
An object of class Seurat 
3000 features across 4460 samples within 1 assay 
Active assay: RNA (3000 features, 0 variable features)


[1] "Var"
[1] "Idents"
An object of class Seurat 
34188 features across 16635 samples within 3 assays 
Active assay: SCT (16424 features, 3000 variable features)
 2 other assays present: RNA, HTO
 2 dimensional reductions calculated: pca, umap50
An object of class Seurat 
34188 features across 9299 samples within 3 assays 
Active assay: RNA (17756 features, 0 variable features)
 2 other assays present: SCT, HTO
 2 dimensional reductions calculated: pca, umap50
[1] "M15"
An object of class Seurat 
34188 features across 9299 samples within 3 assays 
Active assay: RNA (17756 features, 0 variable features)
 2 other assays present: SCT, HTO
 2 dimensional reductions calculated: pca, umap50


[1] "M05"
An object of class Seurat 
34188 features across 9299 samples within 3 assays 
Active assay: RNA (17756 features, 0 variable features)
 2 other assays present: SCT, HTO
 2 dimensional reductions calculated: pca, umap50


[1] "M02"
An object of class Seurat 
34188 features across 9299 samples within 3 assays 
Active assay: RNA (17756 features, 0 variable features)
 2 other assays present: SCT, HTO
 2 dimensional reductions calculated: pca, umap50


...

[1] "M11"
An object of class Seurat 
34188 features across 9299 samples within 3 assays 
Active assay: RNA (17756 features, 0 variable features)
 2 other assays present: SCT, HTO
 2 dimensional reductions calculated: pca, umap50


[1] "M07"
An object of class Seurat 
34188 features across 9299 samples within 3 assays 
Active assay: RNA (17756 features, 0 variable features)
 2 other assays present: SCT, HTO
 2 dimensional reductions calculated: pca, umap50


[1] "M12"
An object of class Seurat 
34188 features across 9299 samples within 3 assays 
Active assay: RNA (17756 features, 0 variable features)
 2 other assays present: SCT, HTO
 2 dimensional reductions calculated: pca, umap50


[1] "M09"
An object of class Seurat 
34188 features across 9299 samples within 3 assays 
Active assay: RNA (17756 features, 0 variable features)
 2 other assays present: SCT, HTO
 2 dimensional reductions calculated: pca, umap50


$T0
An object of class Seurat 
3000 features across 4460 samples within 1 assay 
Active assay: RNA (3000 features, 3000 variable features)

$M15
An object of class Seurat 
3000 features across 1910 samples within 1 assay 
Active assay: RNA (3000 features, 3000 variable features)

$M05
An object of class Seurat 
3000 features across 1434 samples within 1 assay 
Active assay: RNA (3000 features, 3000 variable features)

$M02
An object of class Seurat 
3000 features across 1581 samples within 1 assay 
Active assay: RNA (3000 features, 3000 variable features)

$M10
An object of class Seurat 
3000 features across 1533 samples within 1 assay 
Active assay: RNA (3000 features, 3000 variable features)

$M11
An object of class Seurat 
3000 features across 680 samples within 1 assay 
Active assay: RNA (3000 features, 3000 variable features)

$M07
An object of class Seurat 
3000 features across 1051 samples within 1 assay 
Active assay: RNA (3000 features, 3000 variable features)

$M12
An object 

In [ ]:
%%R
# The violins used to select the cutoffs
for (SAMPLE in names(SEUR)){
    print(SAMPLE)
    # Violin plots. This samples are well filtered
    print(VlnPlot(SEUR[[SAMPLE]], features = c("nFeature_RNA", "nCount_RNA"))); gc()
}

In [ ]:
%%R
### The summary of the changes for both new and old data
#DATA <- readRDS(INPUTfiles[n])
#print("Initial dataset: "); print(DATA)
#DefaultAssay(DATA) <- "RNA"
#print("RNA data: "); print(DATA)
#DATA = subset(DATA, subset = (barcode %ni% c("not.detected")))  # Remove cells no classified
#print("Remove not detected cells: "); print(DATA)
#DATA <- FindVariableFeatures(DATA, selection.method = "vst", nfeatures = 3000)   # To select the features of more interest
#DATA <- CreateSeuratObject(GetAssayData(DATA)[VariableFeatures(DATA),], project = "BESPreducedVarFea", 
                            #meta.data = DATA@meta.data)
#DATA <- FindVariableFeatures(DATA, selection.method = "vst", nfeatures = 3000)   # To set all features as relevant


[1] "Initial dataset: "
An object of class Seurat 
45403 features across 4307 samples within 3 assays 
Active assay: SCT (14346 features, 3000 variable features)
 2 other assays present: RNA, HTO
 5 dimensional reductions calculated: pca, umap_50, tsne_50, umap_elbow, tsne_elbow
[1] "RNA data: "
An object of class Seurat 
45403 features across 4307 samples within 3 assays 
Active assay: RNA (31053 features, 0 variable features)
 2 other assays present: HTO, SCT
 5 dimensional reductions calculated: pca, umap_50, tsne_50, umap_elbow, tsne_elbow
[1] "Remove not detected cells: "
An object of class Seurat 
45403 features across 2521 samples within 3 assays 
Active assay: RNA (31053 features, 0 variable features)
 2 other assays present: HTO, SCT
 5 dimensional reductions calculated: pca, umap_50, tsne_50, umap_elbow, tsne_elbow
